In [12]:
from Crypto.Util.number import getPrime, inverse, long_to_bytes, bytes_to_long, GCD
import random

# RSA

The most famous asymmetric encryption algorithm is RSA, from the authors Rivest, Shamir, Adleman, developed in 1977 and still used in practice today.

RSA is based on exponentiation of integers modulo n, using large integers (typically 2048 bits), exponentiation is easy to compute(requires $O(log{n})^3)$ operations). Nevertheless, security is guaranteed by the cost of factoring large numbers: factoring is difficult (requires $O(e^{log{n}\cdot log{log{n}}})$ operations)

## RSA Key Generation

In order to generate a pair of public/private keys, an user:

- Randomly choose two very large prime numbers: $p,q$

In [13]:
p = getPrime(512)
q = getPrime(512)
print("p = %d" %p)
print("q = %d" %q)

p = 8454184765536934402978499526367255654701447006340963670199776615703794736328236531721815412682282959673539395217982558584118048102404577861531280169369211
q = 12602280893371906279412089437460093951603037864037836859427999056791032168534355544530449108160862975669744748825176022858709961408887033361022771287122533


- Computes module $N = p \cdot q$ and its Euler totien is $\phi(N) = (p-1)\cdot (q-1)$

In [14]:
N = p*q
pIN = (p-1)*(q-1)
print("N = %d" %N)
print("pIN = %d" %pIN)

N = 106542011139761957714063515826040247491132190967454766905360902316127571450500856727160818647965030486356799051690883568192906472818959872753933120364481996248989619514579490941765094244740566854472845540100160512678694371815346190680536841624883854466318370905250471731020562389408023662569298265637874531463
pIN = 106542011139761957714063515826040247491132190967454766905360902316127571450500856727160818647965030486356799051690883568192906472818959872753933120364481975192523960605738808551176130417390960549987975161299630884903021876988441328088460589360363011320383027621106428572439119561398512370958075711586418039720


- Randomly choose public key $e$ such that:

$$ 1 \lt e \lt \phi(N) : gcd(e,\phi(N)) = 1$$

In [20]:
while True:
    e = random.randint(2,pIN)
    if GCD(e,pIN) == 1:
        break
print("e = %d" %e)

e = 93709520440469333662088134280802649470902530273241811884133437203018514699109791967180287214596617194071595369117852054523338053467716938610946597149596872388375941699692439959317059279619560885622894856866354283904995308928361183709647290415085431220209095621172628732376523715108314216071965106727414982047


- Determines private key $d$ as inverse modulo of $e$ in $\phi(n)$:

$$ ed = 1 \mod \phi(N) \implies d = e^{-1} \mod \phi(N) $$



In [25]:
d = inverse(e,pIN)
print("d = %d" %d)

d = 98051411079002250489911719704820297186805963336685239111729262308319212890139566764218608829757013972106302786084885491878392327698350405295036575802906008977089203227634088888314153385849847979188239037182175067824985517210495938079608091102198491909950301952702757855743759375019992858612107797303858040623


- Share public key $\langle N,e \rangle$ and keeps private key $\langle N,d \rangle$.

## RSA Encryption/Decryption

After Alice and Bob generate and share their keys, Alice can encrypt a message $M$ for Bob through his public key $\langle N,e \rangle$:

$$ C = M^e \mod N : M \lt N $$



In [22]:
m = bytes_to_long(b"this is a message from Alice")
c = pow(m,e,N)
print("c = %d" %c)

c = 82980749130567436396322635853788353788608209375517186184165408467267653528158969133143855612296373155596163330605230148626648698332683258778537236754890688764869278981571496454920099635832179184766498397231406530904707385740510513065364361884599938168754357589995007327970926980030540770919264552864462165275


Bob receive ciphertext $C$ that can easily decrypt throught his private key $\langle N,d \rangle$:

$$ M = C^d \mod N$$



In [19]:
m = pow(c,d,N)
long_to_bytes(m)

b'this is a message from Alice'

### Dimonstration:

Choosing $d$ as inverse $e$ in modulo $\phi(N)$ ($ed = 1 \mod \phi(N)$), then:

$$ ed = 1 + k \cdot \phi(N) : k \in \mathbb{N}$$

Hence, using Euler's threorem ($a^{\phi(N)} \mod N = 1$ where $gcd(a,n) = 1$:

$ M = C^d \mod N = $

$ M^{ed} \mod (N)= $

$ M^{1+k\cdot\phi(N)} \mod N = $

$ M \cdot (M^{\phi(N)})^k \mod N = $

$ M \cdot (1)^k \mod N = M $

### Why private key cannot be reversed ?

Choosing $d$ as inverse $e$ in modulo:

$$ d = e^{-1} \mod \phi(N)$$

In order to calculate $d$, it need to known Euler Totien $\phi(N)$, so to calculate it then it need factorise $N$ in $p,q$.

## Fail Encryption

Encryption uses exponentation to power $e$, obliviosly $e$ cannot be an even due to condition $gcd(e,\phi(N)) = 1$, because $\phi(N) = (p-1)*(q-1)$ is surely an even number.

In [28]:
e = random.randint(2,pIN//2)
e *= 2
print("e = %d" %e)
d = inverse(e,pIN)
print("d = %d" %d)
m = bytes_to_long(b"this is a message from Alice")
c = pow(m,e,N)
print("c = %d" %c)
m = pow(c,d,N)
long_to_bytes(m)

e = 38999006503976726131784194848160991191237921312644861593642146509096532993386114850209657571677579161799159907168245754613901595925385611930461105473432951850260352315115278623454687501657695544519670654823542784812289285490378802780070729526374376554729460827447017095347029868911847943570920791861237540276
d = 996782827162893571542703236208591588540972926527337535754955075387658878729844541149779779832936316233243582008012070891918316326959535056382259487199883883670556066654484960451792342377145393683302735817886853042403029972725188979724043807805713854294185415246980041260018497700069231573044818588744922229
c = 27619670001298941370164986820494773726246187109255518179957251262204804381228017770332491065572849719602818196014584480801783687158041847066820795656181021790522535329685257011629061108140187599483266143539030618526227040638181924287144787561787741867743224168395458335099030457397703890178334192034695257574


b'R\x19h9\xea\xfd\xfe\xdc\xf0y74\xaf\xa4T\r2\x88\xcb\xf6+?g\xba\x0f\xae\xf3v=@\xbe\x83\x00\x12\xe93\xde#\x020\xe7:-\xde\xc7\xd3Cv\xa9\xae\xb6:\xd0\xf8\x91%\xc0\xde\xe8\x04\x07\xfd6\xb4&\xfd\xcb\xf4\xf6\xec\xda\xbb\xb7\xd4\xd24\x89\x0f\xa6\xa5\xc7\xe6s\x95\xf1\xca\x82\xe9\x8e\x89\x06\xe3\xa2\x91\x84\x18\r\xa2\xec\x15\x81&\x88\x08/\xf0\xe8\xc07\x04nmi\xf4\xac:Y|\xd5\xd0D\x87W\xb1\x97nj\x11'

If $e$ is not coprime in $\phi(N)$ then encryption and decryption message will fail due to property violation used in demonstration.

## Efficient Encryption

For an efficient RSA encryption, can be setted a fixed public exponent $e$ odd and small, because a small exponent will encrypt faster. 

Nevertheless a very small public exponent $e$ can be subject to attack (Coppersmith & Chinese Remainder Theorem), so usually $e = 65537$.

NOTE: using a fixed public exponent is not a weakness, because in any case public exponent $e$ can be recovered by public key.

In [30]:
e = 65537
print("e = %d" %e)
d = inverse(e,pIN)
print("d = %d" %d)
m = bytes_to_long(b"this is a message from Alice")
c = pow(m,e,N)
print("c = %d" %c)
m = pow(c,d,N)
long_to_bytes(m)

e = 65537
d = 69538345156221947010376838876647871987322878200602372009350635466566319312680991600230465502948016831010148151976403323762936575901109427606535075203178608860036504797449952481446495546086917886472307818859449030345099116349246651646915508947457585932059508468389268385584987705247743513858304294720677352473
c = 79963931948564483299693221552799818091675567761993026319485707120516902205883809942444269626284639280549907531071854431368311443436803440891702006430032909548409032172339215859238126029417783993955642348984352894029599084438112118771896602782124013347420058936190117361693679538677126509867266525927462213369


b'this is a message from Alice'

## RSA Security

Possible approach to attack RSA are:

| Method | Description |
| :----- | :---------- |
| Bruteforce | Infeasible with large prime numbers. |
| Mathematical attacks | Based on computing $\phi(N)$ in order to factorize $N$ |
| Timing Attacks | Based on time of decryption |
| Chosen Ciphertext Attacks | Based on RSA paddings |

### Mathematical Attacks

Security of RSA is based on hardness to factoring large integers, but there are particular condition that simplify problem of factoring:

- Re-using of prime number
- One of prime number is small
- Very close primes numbers

There exists 3 mathematical approach to attack RSA:

- Factorise $N = pq$, in order to compute $\phi(N)$ and then private key $d$
- Determine $\phi(N)$ e then compute private key $d$
- Determine $d$ directly

Today RSA 2048 bits is considered secure, ensuring $p,q$ of similar size and matching constraints.

### Timing Attacks

Paul Kocher in mid 1990' exploted RSA using timing variation in operation, for example multiplying small vs large number, infering on operand size based on time execution.

So there are some contermeasures:

- Use constant exponentiation time
- Add random delays

### Chosen-Ciphertext Attacks

RSA is vulnerable to CCA (**Chosen Ciphertext Attack**), when an attacker can decrypt an arbitrary ciphertext and get plaintet. Attacker can exploit properties of RSA to get info useful for cryptoanalysis. 

In order to avoid it, message can be padded using a secure padding OASP (**Optimal Asymmetric Encryption Padding**), that ensure message is larger enough to be not exploited by CCA attacks.

## CTF Writeups


| Title | Complexity | Points | Arguments |
| :---- | :--------- | :----- | :-------- |
| [Cyber Grabs CTF 0x03 – Unbr34k4bl3](./RSA_CTF/Cyber%20Grabs%20CTF%200x03%20%E2%80%93%20Unbr34k4bl3.ipynb) | a | a | a |

| Title | Complexity | Points | Arguments |
| :- | :- | :- | :- |-
|   | 4.5/5 | 942/1000 | - Multiprime RSA - Rabin Cryptosytem|